# Lesson 07: Understanding Traceback in Python.ipynb

When an exception occurs in a Python program, often a traceback will be printed.
Knowing how to read the traceback can help you easily identify the error and
make a fix. In this tutorial, we are going to see what the traceback can tell
you.

After completing this tutorial, you will know:

*    How to read a traceback
*    How to print the call stack without an exception
*    What is not shown in the traceback


## Tutorial Overview

This tutorial is divided into four parts; they are:

1.    The call hierarchy of a simple program
2.    Traceback upon exception
3.    Triggering traceback manually
4.    An example in model training

The call hierarchy of a simple program

Let’s consider a simple program:

In [23]:
def indentprint(x, indent=0, prefix="", suffix=""):
    if isinstance(x, dict):
        printdict(x, indent, prefix, suffix)
    elif isinstance(x, list):
        printlist(x, indent, prefix, suffix)
    elif isinstance(x, str):
        printstring(x, indent, prefix, suffix)
    else:
        printnumber(x, indent, prefix, suffix)


def printdict(x, indent, prefix, suffix):
    spaces = " " * indent
    print(spaces + prefix + "{")
    for n, key in enumerate(x):
        comma = "," if n != len(x)-1 else ""
        indentprint(x[key], indent+2, str(key)+": ", comma)
    print(spaces + "}" + suffix)


def printlist(x, indent, prefix, suffix):
    spaces = " " * indent
    print(spaces + prefix + "[")
    for n, item in enumerate(x):
        comma = "," if n != len(x)-1 else ""
        indentprint(item, indent+2, "", comma)
    print(spaces + "]" + suffix)


def printstring(x, indent, prefix, suffix):
    spaces = " " * indent
    print(spaces + prefix + '"' + str(x) + '"' + suffix)


def printnumber(x, indent, prefix, suffix):
    spaces = " " * indent
    print(spaces + prefix + str(x) + suffix)


data = {
    "a": [{
        "p": 3, "q": 4,
        "r": [3, 4, 5],
    }, {
        "f": "foo", "g": 2.71
    }, {
        "u": None, "v": "bar"
    }],
    "c": {
        "s": ["fizz", 2, 1.1],
        "t": []
    },
}

indentprint(data)


{
  a: [
    {
      p: 3,
      q: 4,
      r: [
        3,
        4,
        5
      ]
    },
    {
      f: "foo",
      g: 2.71
    },
    {
      u: None,
      v: "bar"
    }
  ],
  c: {
    s: [
      "fizz",
      2,
      1.1
    ],
    t: [
    ]
  }
}


This is a short program, but functions are calling each other. If we add a line
at the beginning of each function, we can reveal how the output is produced with
the flow of control:

In [24]:
def indentprint(x, indent=0, prefix="", suffix=""):
    print(f'indentprint(x, {indent}, "{prefix}", "{suffix}")')
    if isinstance(x, dict):
        printdict(x, indent, prefix, suffix)
    elif isinstance(x, list):
        printlist(x, indent, prefix, suffix)
    elif isinstance(x, str):
        printstring(x, indent, prefix, suffix)
    else:
        printnumber(x, indent, prefix, suffix)


def printdict(x, indent, prefix, suffix):
    print(f'printdict(x, {indent}, "{prefix}", "{suffix}")')
    spaces = " " * indent
    print(spaces + prefix + "{")
    for n, key in enumerate(x):
        comma = "," if n != len(x)-1 else ""
        indentprint(x[key], indent+2, str(key)+": ", comma)
    print(spaces + "}" + suffix)


def printlist(x, indent, prefix, suffix):
    print(f'printlist(x, {indent}, "{prefix}", "{suffix}")')
    spaces = " " * indent
    print(spaces + prefix + "[")
    for n, item in enumerate(x):
        comma = "," if n != len(x)-1 else ""
        indentprint(item, indent+2, "", comma)
    print(spaces + "]" + suffix)


def printstring(x, indent, prefix, suffix):
    print(f'printstring(x, {indent}, "{prefix}", "{suffix}")')
    spaces = " " * indent
    print(spaces + prefix + '"' + str(x) + '"' + suffix)


def printnumber(x, indent, prefix, suffix):
    print(f'printnumber(x, {indent}, "{prefix}", "{suffix}")')
    spaces = " " * indent
    print(spaces + prefix + str(x) + suffix)


indentprint(data)

indentprint(x, 0, "", "")
printdict(x, 0, "", "")
{
indentprint(x, 2, "a: ", ",")
printlist(x, 2, "a: ", ",")
  a: [
indentprint(x, 4, "", ",")
printdict(x, 4, "", ",")
    {
indentprint(x, 6, "p: ", ",")
printnumber(x, 6, "p: ", ",")
      p: 3,
indentprint(x, 6, "q: ", ",")
printnumber(x, 6, "q: ", ",")
      q: 4,
indentprint(x, 6, "r: ", "")
printlist(x, 6, "r: ", "")
      r: [
indentprint(x, 8, "", ",")
printnumber(x, 8, "", ",")
        3,
indentprint(x, 8, "", ",")
printnumber(x, 8, "", ",")
        4,
indentprint(x, 8, "", "")
printnumber(x, 8, "", "")
        5
      ]
    },
indentprint(x, 4, "", ",")
printdict(x, 4, "", ",")
    {
indentprint(x, 6, "f: ", ",")
printstring(x, 6, "f: ", ",")
      f: "foo",
indentprint(x, 6, "g: ", "")
printnumber(x, 6, "g: ", "")
      g: 2.71
    },
indentprint(x, 4, "", "")
printdict(x, 4, "", "")
    {
indentprint(x, 6, "u: ", ",")
printnumber(x, 6, "u: ", ",")
      u: None,
indentprint(x, 6, "v: ", "")
printstring(x, 6, "v: ", "")
     

So now we know the order of how each function is invoked. This is the idea of a
call stack. At any point in time, when we run a line of code in a function, we
want to know what invoked this function.

Traceback upon exception

If we make one typo in the code like the following:

In [25]:
def printdict(x, indent, prefix, suffix):
    spaces = " " * indent
    print(spaces + prefix + "{")
    for n, key in enumerate(x):
        comma = "," if n!=len(x)-1 else ""
        indentprint(x[key], indent+2, str(key)+": ", comma)
    print(spaces + "}") + suffix
    
indentprint(data)

indentprint(x, 0, "", "")
{
indentprint(x, 2, "a: ", ",")
printlist(x, 2, "a: ", ",")
  a: [
indentprint(x, 4, "", ",")
    {
indentprint(x, 6, "p: ", ",")
printnumber(x, 6, "p: ", ",")
      p: 3,
indentprint(x, 6, "q: ", ",")
printnumber(x, 6, "q: ", ",")
      q: 4,
indentprint(x, 6, "r: ", "")
printlist(x, 6, "r: ", "")
      r: [
indentprint(x, 8, "", ",")
printnumber(x, 8, "", ",")
        3,
indentprint(x, 8, "", ",")
printnumber(x, 8, "", ",")
        4,
indentprint(x, 8, "", "")
printnumber(x, 8, "", "")
        5
      ]
    }


TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

The typo is at the last line, where the closing bracket should be at the end of
the line, not before any +. The return value of the print() function is a Python
None object. And adding something to None will trigger an exception.

The lines starting with “Traceback (most recent call last):” are the traceback.
It is the stack of your program at the time when your program encountered the
exception. In the above example, the traceback is in the “most recent call last”
order. Hence your main function is at the top while the one triggering the
exception is at the bottom. So we know the issue is inside the function
printdict().

Usually, you will see the error message at the end of the traceback. In this
example, it is a TypeError triggered by adding None and string. But the
traceback’s help stops here. You need to figure out which one is None and which
one is string. By reading the traceback, we also know the exception-triggering
function printdict() is invoked by indentprint(), and it is in turn invoked by
printlist(), and so on.

The information is essentially the same, but it gives you the lines before and
after each function call.

## Triggering traceback manually

The easiest way to print a traceback is to add a raise statement to manually
create an exception. But this will also terminate your program. If we want to
print the stack at any time, even without any exception, we can do so with the
following:

In [ ]:
import traceback

def printdict(x, indent, prefix, suffix):
    spaces = " " * indent
    print(spaces + prefix + "{")
    for n, key in enumerate(x):
        comma = "," if n!=len(x)-1 else ""
        indentprint(x[key], indent+2, str(key)+": ", comma)
    traceback.print_stack()    # print the current call stack
    print(spaces + "}" + suffix)
    
indentprint(data)

indentprint(x, 0, "", "")
{
indentprint(x, 2, "a: ", ",")
printlist(x, 2, "a: ", ",")
  a: [
indentprint(x, 4, "", ",")
    {
indentprint(x, 6, "p: ", ",")
printnumber(x, 6, "p: ", ",")
      p: 3,
indentprint(x, 6, "q: ", ",")
printnumber(x, 6, "q: ", ",")
      q: 4,
indentprint(x, 6, "r: ", "")
printlist(x, 6, "r: ", "")
      r: [
indentprint(x, 8, "", ",")
printnumber(x, 8, "", ",")
        3,
indentprint(x, 8, "", ",")
printnumber(x, 8, "", ",")
        4,
indentprint(x, 8, "", "")
printnumber(x, 8, "", "")
        5
      ]
    },
indentprint(x, 4, "", ",")
    {
indentprint(x, 6, "f: ", ",")
printstring(x, 6, "f: ", ",")
      f: "foo",
indentprint(x, 6, "g: ", "")
printnumber(x, 6, "g: ", "")
      g: 2.71
    },
indentprint(x, 4, "", "")
    {
indentprint(x, 6, "u: ", ",")
printnumber(x, 6, "u: ", ",")
      u: None,
indentprint(x, 6, "v: ", "")
printstring(x, 6, "v: ", "")
      v: "bar"
    }
  ],
indentprint(x, 2, "c: ", "")
  c: {
indentprint(x, 4, "s: ", ",")
printlist(x

  File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/home/juan/.virtualenvs/dl4cv/lib/python3.8/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/juan/.virtualenvs/dl4cv/lib/python3.8/site-packages/traitlets/config/application.py", line 846, in launch_instance
    app.start()
  File "/home/juan/.virtualenvs/dl4cv/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 677, in start
    self.io_loop.start()
  File "/home/juan/.virtualenvs/dl4cv/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 199, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.8/asyncio/base_events.py", line 570, in run_forever
    self._run_once()
  File "/usr/lib/python3.8/asyncio/base_events.py", line 1859, in _run_once
    handle._run()
  File "/usr/lib/python3.8/asy

The line traceback.print_stack() will print the current call stack.

But indeed, we often want to print the stack only when there is an error (so we
learn more about why it is so). The more common use case is the following:

In [ ]:
import traceback
import random

def compute():
    n = random.randint(0, 10)
    m = random.randint(0, 10)
    return n/m

def compute_many(n_times):
    try:
        for _ in range(n_times):
            x = compute()
        print(f"Completed {n_times} times")
    except:
        print("Something wrong")
        traceback.print_exc()

compute_many(100)

Something wrong


Traceback (most recent call last):
  File "/tmp/ipykernel_158350/2291161378.py", line 12, in compute_many
    x = compute()
  File "/tmp/ipykernel_158350/2291161378.py", line 7, in compute
    return n/m
ZeroDivisionError: division by zero


This is a typical pattern for repeatedly calculating a function, such as Monte
Carlo simulation. But if we are not careful enough, we may run into some error,
such as in the above example where we may have division by zero. The problem is,
in the case of more complicated computations, you can’t easily spot the flaw.
Such as in the above, the issue is buried inside the call to `compute()`.
Therefore, it is helpful to understand how we get the error. But at the same
time, we want to handle the case of the error rather than let the entire program
terminate. If we use the `try-catch` construct, the traceback will not be printed
by default. Therefore, we need to use the `traceback.print_exc()` statement to do
it manually.

Actually, we can have the traceback more elaborated. Because the traceback is
the call stack, we can examine each function in the call stack and check the
variables in each level. In this complicated case, this is the function I
usually use to do a more detailed trace:

In [ ]:
def print_tb_with_local():
    """Print stack trace with local variables. This does not need to be in
    exception. Print is using the system's print() function to stderr.
    """
    import traceback, sys
    tb = sys.exc_info()[2]
    stack = []
    while tb:
        stack.append(tb.tb_frame)
        tb = tb.tb_next()
    traceback.print_exc()
    print("Locals by frame, most recent call first", file=sys.stderr)
    for frame in stack:
        print("Frame {0} in {1} at line {2}".format(
            frame.f_code.co_name,
            frame.f_code.co_filename,
            frame.f_lineno), file=sys.stderr)
        for key, value in frame.f_locals.items():
            print("\t%20s = " % key, file=sys.stderr)
            try:
                if '__repr__' in dir(value):
                    print(value.__repr__(), file=sys.stderr)
                elif '__str__' in dir(value):
                    print(value.__str__(), file=sys.stderr)
                else:
                    print(value, file=sys.stderr)
            except:
                print("", file=sys.stderr)

## An example of model training

The call stack, as reported in the traceback, has a limitation: You can only see
the Python functions. It should be just fine for the program you wrote, but many
large libraries in Python have part of them written in another language and
compiled into binary. An example is Tensorflow. All the underlying operations
are in binary for the performance. Hence if you run the following code, you will
see something different:

In [ ]:
import numpy as np

sequence = np.arange(0.1, 1.0, 0.1)  # 0.1 to 0.9
n_in = len(sequence)
sequence = sequence.reshape((1, n_in, 1))

# define model
import tensorflow as tf
from tensorflow.keras.layers import LSTM, RepeatVector, Dense, TimeDistributed, Input
from tensorflow.keras import Sequential, Model

model = Sequential([
    LSTM(100, activation="relu", input_shape=(n_in+1, 1)),
    RepeatVector(n_in),
    LSTM(100, activation="relu", return_sequences=True),
    TimeDistributed(Dense(1))
])
model.compile(optimizer="adam", loss="mse")

model.fit(sequence, sequence, epochs=300, verbose=0)

2022-07-19 13:55:31.738690: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-07-19 13:55:31.738824: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-07-19 13:55:31.738916: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2022-07-19 13:55:31.739003: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcurand.so.10'; dlerror: libcurand.so.10: cannot open shared object file: No such file or directory
2022-07-19 13:55:31.739087: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

ValueError: in user code:

    File "/home/juan/.virtualenvs/dl4cv/lib/python3.8/site-packages/keras/engine/training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "/home/juan/.virtualenvs/dl4cv/lib/python3.8/site-packages/keras/engine/training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/juan/.virtualenvs/dl4cv/lib/python3.8/site-packages/keras/engine/training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "/home/juan/.virtualenvs/dl4cv/lib/python3.8/site-packages/keras/engine/training.py", line 808, in train_step
        y_pred = self(x, training=True)
    File "/home/juan/.virtualenvs/dl4cv/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/juan/.virtualenvs/dl4cv/lib/python3.8/site-packages/keras/engine/input_spec.py", line 

If you look at the traceback, you cannot really see the complete call stack. For
example, the top frame you know you called model.fit(), but the second frame is
from a function named error_handler(). Here, you cannot see how the fit()
function triggered that. This is because Tensorflow is highly optimized. A lot
of stuff is hidden in compiled code and not visible by the Python interpreter.

In this case, it is essential to patiently read the traceback and find the clue
to the cause. Of course, the error message usually should also give you some
useful hints.

## Further Reading

This section provides more resources on the topic if you are looking to go
deeper.

### Books

*    [Python Cookbook, 3rd edition](https://www.amazon.com/dp/1449340377) by David Beazley and Brian K. Jones

### Python Official Documentation

* [The traceback module](https://docs.python.org/3/library/traceback.html) in Python standard library

## Summary

In this tutorial, you discovered how to read and print the traceback from a
Python program.

Specifically, you learned:

*    What information the traceback tells you
*    How to print a traceback at any point of your program without raising an exception
